In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_curve, recall_score, auc, roc_auc_score

!ls ../results

1.jpg
acc_features_for_all_attributes.npy
berk.csv
berk-test.csv
GoogleLenetv2.png
GoogLeNet_binary61_feature_imagenet_all.npy
GoogLeNet_binary61_feature_imagenet_test7600.npy
googlenet_weights.npy
logs
metrics
models
myRAP_labels_pd.csv
OEDCWPAL_PETA_features_all.pickle
orig_feature_maps_dataset_weighted_by_label.npy
orig_feature_maps_dataset_weighted_by_predprob.npy
orig_feature_maps_trainset_weighted_by_label.npy
orig_feature_maps_trainset_weighted_by_predprob.npy
PA-100K_labels_pd.csv
partGoogleLenet.png
PETA_coarse_to_fine_labels_pd.csv
PETA.csv
PETA_labels_pd.csv
PETA_lowerBody_labels_pd.csv
PETA_ratio_positive_samples_for_attributes.json
predictions
RAP_labels_pd.csv
RAP_partion.npy
RAP_partion.txt
README.md
relation_array.npy
resize_feature_maps_dataset_weighted_by_label.npy
resize_feature_maps_dataset_weighted_by_predprob.npy
resize_feature_maps_trainset_weighted_by_label.npy
resize_feature_maps_trainset_weighted_by_predprob.npy
samples
state_transition_matrix.npy
test_predict

# Preparing attributes_list & labels

In [2]:
import scipy.io as sio

data = sio.loadmat('/home/anhaoran/data/pedestrian_attributes_RAP/RAP_annotation/RAP_annotation.mat')['RAP_annotation']
attributes_list = []
for i in range(data['attribute_eng'][0][0].shape[0]):
    attributes_list.append(data['attribute_eng'][0][0][i][0][0])
print(attributes_list)
attributes_list_chinese = []
for i in range(data['attribute_chinese'][0][0].shape[0]):
    attributes_list_chinese.append(data['attribute_chinese'][0][0][i][0][0])
print(attributes_list_chinese)

['Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'hs-BaldHead', 'hs-LongHair', 'hs-BlackHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand', 'faceFront', 'faceBack', 'faceLeft', 'faceRight', 'occlusionLeft', 'occlusionRight', 'occlusionUp', 'occlusionDown', 'occlusion-Environment', 'occlusion-Attachment', 'occlusion-Person', 'occlusion-Other', 'up-Black

In [3]:
low_level = [11]
mid_level = [9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42]
high_level = [0,1,2,3,4,5,6,7,8,43,44,45,46,47,48,49,50]
attributes_list_hiar = list(np.array(attributes_list)[list(np.hstack((low_level, mid_level, high_level)))])
print("The total attributes labeled: ", len(attributes_list_hiar))
print(attributes_list_hiar)
low_level_new = [11]#,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91
mid_level_new = [4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50]
high_level_new = [0,1,2,3]#,51,52,53,54,55,56,57,58,59,60,61,62
attributes_list_hiar_new = list(np.array(attributes_list)[list(np.hstack((low_level_new, mid_level_new, high_level_new)))])
print(attributes_list_hiar_new)

The total attributes labeled:  51
['hs-BlackHair', 'hs-BaldHead', 'hs-LongHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 'ub-TShirt', 'ub-Cotton', 'ub-Jacket', 'ub-SuitUp', 'ub-Tight', 'ub-ShortSleeve', 'lb-LongTrousers', 'lb-Skirt', 'lb-ShortSkirt', 'lb-Dress', 'lb-Jeans', 'lb-TightTrousers', 'shoes-Leather', 'shoes-Sport', 'shoes-Boots', 'shoes-Cloth', 'shoes-Casual', 'attach-Backpack', 'attach-SingleShoulderBag', 'attach-HandBag', 'attach-Box', 'attach-PlasticBag', 'attach-PaperBag', 'attach-HandTrunk', 'attach-Other', 'Female', 'AgeLess16', 'Age17-30', 'Age31-45', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'action-Calling', 'action-Talking', 'action-Gathering', 'action-Holding', 'action-Pusing', 'action-Pulling', 'action-CarrybyArm', 'action-CarrybyHand']
['hs-BlackHair', 'BodyFat', 'BodyNormal', 'BodyThin', 'Customer', 'Clerk', 'hs-BaldHead', 'hs-LongHair', 'hs-Hat', 'hs-Glasses', 'hs-Muffler', 'ub-Shirt', 'ub-Sweater', 'ub-Vest', 

In [4]:
from keras.preprocessing import image
import tqdm

filename = r"../results/RAP_labels_pd.csv"
data = np.array(pd.read_csv(filename))[:, 1:]
length = len(data)
image_height = 160
image_width = 75
data_x = np.zeros((length, image_height, image_width, 3))
data_y = np.zeros((length, 51))
for i in tqdm.tqdm(range(length)):
    img = image.load_img(data[i, 0], target_size=(image_height, image_width, 3))
    data_x[i] = image.img_to_array(img)
    data_y[i] = np.array(data[i, 1:52], dtype="float32")
data_y_hiar = data_y[:, list(np.hstack((low_level, mid_level, high_level)))]
data_y_hiar_new = data_y[:, list(np.hstack((low_level_new, mid_level_new, high_level_new)))]
split = np.load('../results/RAP_partion.npy').item()
X_train = data_x[list(split['train'][0])]
y_train = data_y[list(split['train'][0])]
y_train_hiar = data_y_hiar[list(split['train'][0])]
y_train_hiar_new = data_y_hiar_new[list(split['train'][0])]
X_test = data_x[list(split['test'][0])]
y_test = data_y[list(split['test'][0])]
y_test_hiar = data_y_hiar[list(split['test'][0])]
y_test_hiar_new = data_y_hiar_new[list(split['test'][0])]
print("The shape of the X_train is: ", X_train.shape)
print("The shape of the X_test is: ", X_test.shape)
print("The shape of the y_train is: ", y_train.shape)
print("The shape of the y_test is: ", y_test.shape)

Using TensorFlow backend.
100%|██████████| 41585/41585 [01:31<00:00, 455.94it/s]


The shape of the X_train is:  (33268, 160, 75, 3)
The shape of the X_test is:  (8317, 160, 75, 3)
The shape of the y_train is:  (33268, 51)
The shape of the y_test is:  (8317, 51)


# Predict

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ""
import sys
sys.path.append("..")

In [6]:
from src.network.hiarBayesGoogLenet import hiarBayesGoogLeNet

class_num = len(attributes_list)
model = hiarBayesGoogLeNet.build(image_height, image_width, 3, [len(low_level_new), len(mid_level_new), len(high_level_new)])
model.load_weights("../models/imagenet_models/hiarBayesGoogLeNet_RAP/binary51_newhier_newlossnoexp_split0_epoch100_valloss0.39.hdf5")
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 75, 3)   0                                            
__________________________________________________________________________________________________
conv1_7x7_s2 (Conv2D)           (None, 80, 38, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
conv1_7x7_s2_bn (BatchNormaliza (None, 80, 38, 64)   256         conv1_7x7_s2[0][0]               
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 40, 19, 64)   0           conv1_7x7_s2_bn[0][0]            
__________________________________________________________________________________________________
conv2_3x3 

In [7]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)
print("The shape of the pred_train is: ", pred_train.shape)
print("The shape of the pred_test is: ", pred_test.shape)

The shape of the pred_train is:  (33268, 51)
The shape of the pred_test is:  (8317, 51)


# Defining the function

In [8]:
def mA(y_pred, y_true):
    M = len(y_pred)
    L = len(y_pred[0])
    res = 0
    for i in range(L):
        P = sum(y_true[:, i])
        N = M - P
        TP = sum(y_pred[:, i]*y_true[:, i])
        TN = list(y_pred[:, i]+y_true[:, i] == 0.).count(True)
        #print(TP, P, TN, N)
        #print(P,',', N,',', TP,',', TN)
        #if P != 0:
        res += TP/P + TN/N
        """else:
            res += 1 + TN/N"""
    return res / (2*L)

def acc(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        #print(np.shape(y_pred[i]*y_true[i]))
        if sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i]))
            #print(y_pred[i])
            #print(y_true[i])
            #print(sum(y_pred[i]*y_true[i]) / (sum(y_pred[i])+sum(y_true[i])-sum(y_pred[i]*y_true[i])))
            M_ += 1
    #print(M_)
    return res / M_

def prec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_pred[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_pred[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

def rec(y_pred, y_true):
    M = len(y_pred)
    M_ = 0
    res = 0
    for i in range(M):
        if sum(y_true[i]) != 0:
            res += sum(y_pred[i]*y_true[i]) / sum(y_true[i])
            M_ += 1
    if M_ == 0:
        return 0
    return res / M_

# Metrics

In [9]:
predictions = np.array(pred_test >= 0.5, dtype="float64")
label = y_test_hiar_new
print("hiarBayesGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

hiarBayesGoogLeNet
0.7882162781410097
0.450285143080982
0.5055357274986886
0.7858456751056142
0.6152683696157343


In [10]:
predictions = np.array(pred_train >= 0.5, dtype="float64")
label = y_train_hiar_new
print("hiarBayesGoogLeNet")
print(mA(predictions, label))
print(acc(predictions, label))
prec_value = prec(predictions, label)
print(prec_value)
rec_value = rec(predictions, label)
print(rec_value)
print(2*prec_value*rec_value/(prec_value+rec_value))

hiarBayesGoogLeNet
0.8528662512903087
0.4778102030362154
0.5285454983118162
0.8163865720601677
0.6416642997073784


# Samples

In [15]:
hard_indexs = []###the index based on the data_x
hard_pred = []
hard_label = []
train_size = len(X_train)
for i in tqdm.tqdm(range(len(pred_test))):
    pred = np.array(pred_test[i] >= 0.5, dtype="float64").reshape((1, -1))
    lab = y_test_hiar_new[i].reshape((1, -1))
    #print(acc(pred, lab), prec(pred, lab))
    if acc(pred, lab) < 0.45 and prec(pred, lab) < 0.5:
        hard_indexs.append(split['test'][0][i])
        hard_pred.append(pred)
        hard_label.append(lab)
    
#print("-------------------------------")
for i in tqdm.tqdm(range(len(pred_train))):
    pred = np.array(pred_train[i] >= 0.5, dtype="float64").reshape((1, -1))
    lab = y_train_hiar_new[i].reshape((1, -1))
    #print(acc(pred, lab), prec(pred, lab))
    if acc(pred, lab) < 0.45 and prec(pred, lab) < 0.5:
        hard_indexs.append(split['train'][0][i])
        hard_pred.append(pred)
        hard_label.append(lab)
print(len(hard_indexs))

100%|██████████| 33268/33268 [00:03<00:00, 9931.88it/s] 

16630


In [1]:
import matplotlib.pyplot as plt

idx = 0
for i in hard_indexs:
    plt.imshow(image.load_img(data[i, 0], target_size=(image_height, image_width, 3)))
    plt.show()
    l_tmp = data_y_hiar_new[i].reshape((1, -1))
    if i in list(split['test'][0]):
        p_tmp = pred_test[list(split['test'][0]).index(i)].reshape((1, -1))
    else:
        p_tmp = pred_train[list(split['train'][0]).index(i)].reshape((1, -1))
    p_tmp_b = np.array(p_tmp >= 0.5, dtype="float64")
    for j in range(len(p_tmp_b[0])):
        print("[", attributes_list_hiar_new[j], "]", l_tmp[0][j], "/", p_tmp_b[0][j], end="")
    #print(hard_pred[idx]==p_tmp_b, '\t', hard_label[idx]==l_tmp)
    idx+=1

NameError: name 'hard_indexs' is not defined